<a href="https://colab.research.google.com/github/z-guard/analysis/blob/main/notebooks/navitime_location.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q geocoder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 3.0 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
DIR_NAME = '/content/drive/MyDrive/z-gard/data'

In [41]:
import os
import requests
import json
import pandas as pd
from urllib.parse import urlparse, quote
import geocoder
import math
import numpy as np
import glob

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [35]:
def number_to_kanji(str):
    return str.translate(str.maketrans({
        '１':'一', '２':'二', '３':'三', '４':'四', '５':'五', '６':'六', '７':'七', '８':'八', '９':'九',
        '1':'一', '2':'二', '3':'三', '4':'四', '5':'五', '6':'六', '7':'七', '8':'八', '9':'九'
    }))

def modify_address(address):
    address = address.split('-')[0]
    address = address.split('丁目')[0]
    return number_to_kanji(address) + '丁目'

# 緯度経度取得
def add_lat_lon(df_master):
    lats = []
    lons = []
    for i, row in df_master.iterrows():
        if (i % 500) == 0:
            print(f'row={i}')
        address = modify_address(row['住所'])
        ret = geocoder.osm(address, timeout=10)
        if not ret.ok:
            print(i, row['名称'], address)
        lats.append(ret.lat)
        lons.append(ret.lng)
    df_copy = df_master.copy()
    df_copy['緯度'] = lats
    df_copy['経度'] = lons
    return df_copy

# 距離計算
def get_distance(lat1, lon1, lat2, lon2):
    _lat1 = lat1 * math.pi / 180
    _lon1 = lon1 * math.pi / 180
    _lat2 = lat2 * math.pi / 180
    _lon2 = lon2 * math.pi / 180
    _tmp = math.cos(_lat1) * math.cos(_lat2) * math.cos(_lon2 - _lon1) + math.sin(_lat1) * math.sin(_lat2)
    _tmp = 1 if _tmp > 1 else -1 if _tmp < -1 else _tmp
    return 6371 * math.acos(_tmp)

In [10]:
df_navitime = pd.DataFrame()
for filename in glob.glob(os.path.join(DIR_NAME, 'navitime/*.csv')):
    _tmp = pd.read_csv(filename, dtype='str')
    filebody = os.path.splitext(os.path.basename(filename))[0]
    print(filebody, _tmp.shape)
    _tmp['origin'] = filebody
    df_navitime = pd.concat([df_navitime, _tmp])

df_navitime = df_navitime.reset_index(drop=True)
print(df_navitime.shape)

navitime_shopping_street (133, 13)
navitime_spa (540, 16)
navitime_karaoke (750, 26)
navitime_netcafe (186, 19)
navitime_music_school (750, 13)
navitime_dance_school (715, 13)
navitime_fitness (750, 15)
navitime_tennis_school (46, 13)
navitime_golf (294, 17)
navitime_english (700, 13)
navitime_cleaning (750, 17)
navitime_police (750, 9)
navitime_sports_school (748, 13)
navitime_cram_school (750, 10)
(7862, 27)


In [24]:
df_address = df_navitime[['spot-name-text', 'spot-detail-value-text', 'spot-tag', 'spot-tag 2', 'origin']].copy()
df_address.columns = ['名称', '住所', 'タグ1', 'タグ2', 'データ元']

In [36]:
df_address_w_lat_lon = add_lat_lon(df_address)

row=0
0 砂町銀座商店街 東京都江東区北砂丁目
6 かっぱ橋道具街(R) 東京都台東区松が谷一から四丁目
11 柴又帝釈天参道商店街 柴又神明会 東京都葛飾区柴又丁目
13 新大久保コリアンタウン 東京都新宿区大久保・百人町丁目
18 高橋商店街(高橋のらくろード) 東京都江東区高橋丁目
19 中延商店街(なかのぶスキップロード) 東京都品川区中延、東中延丁目
22 吉祥寺パークロード 東京都武蔵野市吉祥寺南町丁目
26 吉祥寺サンロード商店街 東京都武蔵野市吉祥寺本町丁目
34 豪徳寺商店街 東京都世田谷区豪徳寺丁目
41 吉祥寺中道通り 東京都武蔵野市吉祥寺本町丁目
42 阿佐谷パールセンター商店街 東京都杉並区阿佐谷南丁目
44 亀有銀座商店街 ゆうろーど亀有 東京都葛飾区亀有丁目
45 関三通り商店街 東京都足立区丁目
47 高松町商店街 東京都立川市丁目
49 駒込銀座商店街 東京都豊島区丁目
51 高円寺パル商店街 東京都杉並区丁目
52 鳩の街通り商店街 東京都墨田区向島丁目
53 学芸大学東口商店街 東京都目黒区丁目
60 上板南口銀座商店街 東京都板橋区丁目
61 佐竹商店街 東京都台東区丁目
62 雑色商店街 東京都大田区丁目
64 志村銀座商店街 東京都板橋区丁目
65 上原銀座商店街 東京都渋谷区丁目
66 新中山道商店街 東京都板橋区丁目
69 上井草商店街 東京都杉並区丁目
70 上原駅前商店街 東京都渋谷区丁目
71 上板橋北口商店街 東京都板橋区丁目
73 常盤台銀座商店街 東京都板橋区丁目
74 新小岩小松通り商店街 東京都葛飾区丁目
76 西原商店街 東京都渋谷区丁目
77 石川台希望ヶ丘商店街 東京都大田区丁目
78 千住緑町商店会 東京都足立区丁目
81 諏訪通り商店街 東京都立川市丁目
84 人形町商店街 東京都中央区丁目
86 西武井荻商店街 東京都杉並区丁目
87 西立商店街 東京都立川市丁目
89 洗足商店街 東京都目黒区丁目
91 梅島駅前通り商店街 東京都足立区丁目
92 東松中央通り商店街 東京都江戸川区丁目
93 東和銀座商店街 東京都足立区丁目
94 飯田橋商店街 東京都千代田区飯田橋丁目
95 尾山台商栄会商店街 東京都世田谷区尾山台丁目
96 巣鴨駅前商店街 東京都豊島区丁目
98 池袋平和通り商店

In [40]:
YOUR_API_KEY = ''

In [42]:
def get_place_info(place_name, unique_no, is_rating=True):
    place = quote(place_name)
    rating = '%2Crating' if is_rating else ''
    
    url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={place}&inputtype=textquery&fields=name{rating}%2Cgeometry&key={YOUR_API_KEY}"
    response = requests.get(url)
    if response.status_code == requests.codes.ok:
        try:
            _df = pd.DataFrame(response.json()['candidates'])
            _df['no'] = unique_no
            _df['place_name'] = place_name
            _df['lat'] = _df['geometry'].apply(lambda x: x['location']['lat'])
            _df['lng'] = _df['geometry'].apply(lambda x: x['location']['lng'])
            if not is_rating:
                _df['rating'] = np.NaN
            return _df[['no', 'place_name', 'name', 'lat', 'lng', 'rating']]
        except Exception as e:
            print(unique_no, place_name, e)
            pass
    else:
        print(unique_no, place_name, response.status_code)
    
    return pd.DataFrame([{'no': unique_no, 'place_name': place_name}])

In [51]:
df_address_w_lat_lon[df_address_w_lat_lon['緯度'].isna()].head()

,名称,住所,タグ1,タグ2,データ元,緯度,経度
0,砂町銀座商店街,東京都江東区北砂,#商店街,NaN,navitime_shopping_street,NaN,NaN
6,かっぱ橋道具街(R),東京都台東区松が谷1から4丁目・西浅草1から3丁目・寿2丁目・元浅草4丁目にかけて,#商店街,#田原町(東京都)駅,navitime_shopping_street,NaN,NaN
11,柴又帝釈天参道商店街 柴又神明会,東京都葛飾区柴又,#商店街,NaN,navitime_shopping_street,NaN,NaN
13,新大久保コリアンタウン,東京都新宿区大久保・百人町,#商店街,#新大久保駅,navitime_shopping_street,NaN,NaN
18,高橋商店街(高橋のらくろード),東京都江東区高橋,#商店街,NaN,navitime_shopping_street,NaN,NaN


In [54]:
df_result = pd.DataFrame()
for index, row in df_address_w_lat_lon.iterrows():
    if np.isnan(row['緯度']):
        keyword = f"{row['名称']} {row['住所']}"
        _df = get_place_info(keyword, index, is_rating=False)
        df_result = pd.concat([df_result, _df])
df_result = df_result.reset_index(drop=True)
print(df_result.shape)

1127 ポイント大山ハッピーロード店 東京都板橋区大山東町59-2 武田商事ビル2F 'geometry'
1510 まんが図書室l@rk(ラーク) 東京都西東京市ひばりヶ丘北3-5-6 シティコートひばりヶ丘3F 'geometry'
1668 エレキギター教室南町田 東京都町田市鶴間688-3 フロレゾン203号室 'geometry'
4559 サクラメント 東京都八王子市みなみ野2-16-9 'geometry'
4964 エンパイアー 蓮沼店 東京都板橋区蓮沼町80-3 カーサーコモドK 102 'geometry'
5142 ホワイト急便 めじろ台店 東京都八王子市めじろ台1-9-1 'geometry'
6414 PRGR(プロギア) サイエンス・フィットスク-ル昭和の森アカデミ-校 東京都昭島市つつじヶ丘1-1-7 昭和の森ゴルフアカデミ- 'geometry'
6471 クーバー・コーチング・ジャパン・サッカースクール 北八王子校 東京都八王子市石川町1401-1 オーチャードグリーンクラブ 'geometry'
(723, 6)


In [62]:
df_result.head()

,no,place_name,name,lat,lng,rating
0,0,砂町銀座商店街 東京都江東区北砂,Sunamachi Ginza Shopping District,35.679703,139.831326,NaN
1,6,かっぱ橋道具街(R) 東京都台東区松が谷1から4丁目・西浅草1から3丁目・寿2丁目・元浅草4...,Kappabashi Dougu Street,35.714299,139.788889,NaN
2,11,柴又帝釈天参道商店街 柴又神明会 東京都葛飾区柴又,Taishakuten Sandō,35.757675,139.876562,NaN
3,13,新大久保コリアンタウン 東京都新宿区大久保・百人町,新大久保コリアンタウン,35.701353,139.702005,NaN
4,18,高橋商店街(高橋のらくろード) 東京都江東区高橋,高橋のらくろード（高橋商店街）,35.685335,139.799526,NaN


In [73]:
df_merge = pd.merge(
    df_address_w_lat_lon.reset_index().rename(columns={'index': 'no'}),
    df_result.drop_duplicates(subset=['no', 'place_name'])[['no', 'lat', 'lng']],
    on='no', how='left'
)
print(df_merge.shape)
assert len(df_address_w_lat_lon) == len(df_merge)

(7862, 10)


In [74]:
df_merge.loc[df_merge['緯度'].isna(), '緯度'] = df_merge['lat']
df_merge.loc[df_merge['経度'].isna(), '経度'] = df_merge['lng']
df_merge = df_merge.dropna(subset=['緯度', '経度']).drop(columns=['no', 'lat', 'lng'])
print(df_merge.shape)

(7854, 7)


In [84]:
data_source_name = {
    'navitime_shopping_street': '商店街',
    'navitime_spa': '銭湯',
    'navitime_karaoke': 'カラオケ',
    'navitime_netcafe': 'ネットカフェ',
    'navitime_music_school': '音楽教室',
    'navitime_dance_school': 'ダンス教室',
    'navitime_fitness': 'フィットネス',
    'navitime_tennis_school': 'テニス教室',
    'navitime_golf': 'ゴルフ練習場',
    'navitime_english': '英会話',
    'navitime_cleaning': 'クリーニング',
    'navitime_police': '警察署',
    'navitime_sports_school': 'スポーツ教室',
    'navitime_cram_school': '学習塾',
}

In [87]:
df_merge['タグ1'] = df_merge['タグ1'].str.replace('#', '')
df_merge['タグ2'] = df_merge['タグ2'].str.replace('#', '')
df_merge['データ元'] = df_merge['データ元'].map(data_source_name)

In [88]:
df_merge

,名称,住所,タグ1,タグ2,データ元,緯度,経度
0,砂町銀座商店街,東京都江東区北砂,商店街,NaN,商店街,35.679703,139.831326
1,谷中ぎんざ,東京都台東区谷中3-荒川区西日暮里3,商店街,NaN,商店街,35.726835,139.765835
2,日暮里繊維街,東京都荒川区東日暮里3丁目-6丁目,商店街,NaN,商店街,35.730514,139.781233
3,麻布十番商店街,東京都港区麻布十番2丁目,商店街,NaN,商店街,35.655190,139.736334
4,谷中銀座商店街,東京都台東区谷中3,商店街,千駄木駅,商店街,35.726835,139.765835
...,...,...,...,...,...,...,...
7857,公文式松が丘教室,東京都中野区松が丘2丁目27-1 松が丘片山会館,NaN,NaN,学習塾,35.720725,139.670657
7858,公文式文化の森前教室,東京都大田区中央1丁目21-3-101,NaN,NaN,学習塾,35.581180,139.719875
7859,公文式瑞穂西松原教室,東京都西多摩郡瑞穂町箱根ケ崎西松原13-14,NaN,NaN,学習塾,35.768895,139.337727
7860,公文式高円寺中央公園教室,東京都杉並区高円寺南4丁目28-10 高円寺リリエンハイム3F-305,NaN,NaN,学習塾,35.704170,139.652750


In [90]:
df_merge.to_csv(os.path.join(DIR_NAME, 'navitime_master.csv'), index=False, encoding='utf-8_sig')